In [43]:
from modeling.gen6.api import symbolic, get_constraints
from engine.torchengine import EliminateAnalysis, ParallelResiduals, ElimResidual, ipoptsolver
from functools import partial
from engine.torchdata import load_vals, print_formatted_table
import torch
import uuid, base64

def generate_short_id():
    # Generate a UUID
    full_uuid = uuid.uuid4()
    # Convert UUID to bytes and encode in base64
    uuid_bytes = full_uuid.bytes
    encoded = base64.urlsafe_b64encode(uuid_bytes)
    # Take first 8 characters and remove any base64 padding
    short_id = encoded[:8].decode('ascii').rstrip('=')
    return short_id

In [ ]:
# need to have identifiers to sets 
# then architectures point to sets instances (or architectures) through constraints/elimination/parallel
# then set instances (a set with a specific choice of representation function) are built; this is because we need to generate the indices set that is consistent for all set instances (i.e. dimensions)
# this allows for flexibility in building, and efficicency when computing (i.e. we have fixed the dimension of the problem)

In [77]:
def build(architecture, sets, indices):
    # sets_default, indices_default = self.get_sets()
    # if sets is None:
    #     sets = sets_default
    # if indices is None:
    #     indices = indices_default
    if not architecture.parallel and not architecture.elim:
        if len(architecture.constraints) == 1:
            cid, _ = architecture.constraints[0]
            return sets[cid].analysis
        else:
            return EliminateAnalysis([sets[cid].analysis for cid,_ in architecture.constraints])
    else:
        if architecture.parallel:
            parallel_sets = [build(mfs, sets, indices) for mfs in architecture.parallel]
            T = ParallelResiduals(analyses=parallel_sets, functions=[])
            R = EliminateAnalysis([build(mfs, sets, indices) for mfs in architecture.elim], [T], flatten=True)
            solvefor = torch.tensor([p.structure[1] for p in parallel_sets])
            bnds = [(None, None) for _ in solvefor]
            ipsolver = partial(ipoptsolver, bnds_problem=bnds)
            S = ElimResidual(R, solvefor, indices, 
                    solver=ipsolver,
                    solvefor_raw=True)
            return S
        else:
            return EliminateAnalysis([build(mfs, sets, indices) for mfs in architecture.elim])

In [80]:
class MFunctionalSet():
    def __init__(self, *constraints):
        self.constraints = [(generate_short_id(), c) for c in constraints] if constraints is not None else []
        self.elim = []
        self.parallel = []

    def functionalsubsetof(self, *constraints):
        self.constraints += constraints
        return self
    
    def gather_sets(self):
        all_constraints = []
        constraints = list(self.constraints) #deep copy
        counter = 0
        while constraints:
            counter += 1
            c = constraints.pop()
            if hasattr(c, "constraints"):
                constraints += c.constraints
            else:
                all_constraints.append(c)
        sets, _, _, indices = get_constraints(all_constraints)
        return sets, indices

    def config(self, elim=None, parallel=None):
        if elim is None and parallel is None:
            return self
        MFS = MFunctionalSet(self.constraints)
        MFS.elim = elim if elim is not None else []
        MFS.parallel = parallel if parallel is not None else []
        return MFS
    
    
    def build(self, sets=None, indices=None):
        return build(self, sets, indices)

In [81]:
#x, y, z, y1, y2, y3, z1, z2 = symbolic("x", "y", "z", "y1", "y2", "y3", "z1", "z2")
x, y, z, a, b, c, d = symbolic("x", "y", "z", "a", "b", "c", "d")

S1 = MFunctionalSet(x + y == a)
S2 = MFunctionalSet(d == 2*a)
S3 = MFunctionalSet(x == a+d)
D1 = MFunctionalSet().functionalsubsetof(S1, S2, S3)
S4 = MFunctionalSet(c == a+b)
D2 = MFunctionalSet().functionalsubsetof(D1, S4)

set_info = D2.gather_sets()
sets, indices = set_info
# MD = D2.config(elim=[D1.config(elim=[S1], parallel=[S2]), S3]).build()

In [82]:
x0 = {'x': 1, 'y': 2, 'b': 3}
x = load_vals(x0, indices, isdict=True)

In [65]:
aD2a = D2.config(elim=[D1.config(parallel=[S1, S2, S3]), S4]).build(*set_info)
xfp = aD2a(x)
xfp

tensor([-2.,  2.,  3., -1., -3.,  2.], dtype=torch.float64)

In [42]:
aD2b = D2.config(elim=[D1.config(elim=[S1, S2, S3]), S4]).build(*set_info)
aD2b(xfp)

tensor([-2.,  2.,  0., -1., -3., -1.], dtype=torch.float64)

In [66]:
idxrev = {var.item():key for key,var in indices.items()}
print_formatted_table([xfp], indices, idxrev)

d  y b a  x  c
-2 2 3 -1 -3 2


In [33]:
aD1 = D1.config(elim=[S1, S2]).build(sets)
aD1(x)

tensor([1., 3., 2., 0., 2.], dtype=torch.float64)

In [ ]:
s, indices = D2.get_sets()
A1 = S1.build(s, indices)

In [ ]:
# F1: Instantiate directly from config: C1.config(eliminate=[C2.config(),D1.config()], parallel=[C4,C5])
# F2: store information: C1.children = [C2,D1,C4,C5]; A1.children=[C5,C6]
# F3: preprocess: e.g. C1.flatten() -> new store information

In [83]:
A,V,S,W,Ww,CL,Re,Cf,CD,D = symbolic('A','V','S','W','Ww','C_L','Re','C_f','C_D','D')
rho,mu,k,t,e,CDA,SwetS,W0,pi = 1.23, 1.78e-5, 1.2, 0.12, 0.96, 0.0306, 2.05, 4940, 3.14

Aero = MFunctionalSet(
    CL == 2*W/(rho*V**2*S),
    Re == rho*V/mu*(S/A)**(1/2),
    Cf == 0.074/Re**0.2,
    CD == CDA/S+k*Cf*SwetS+CL**2/(3.14*A*e),
    D == 0.5*rho*V**2*CD*S
)
Struct1 = MFunctionalSet(
    Ww == 45.42*S+8.71e-5*2.5/t*A**(3/2)*S**(1/2)*(W0*W)**(1/2),
)
Struct2 = MFunctionalSet(
    W == W0+Ww
)

Struct = MFunctionalSet().functionalsubsetof(Struct1, Struct2)
AeroStruct = MFunctionalSet().functionalsubsetof(Aero, Struct1, Struct2)

In [84]:
set_info = AeroStruct.gather_sets()
sets, indices = set_info

In [156]:
x0 = {'A': 18.16, 'V': 49.18, 'S': 5.256, 'W':7000}
x = load_vals(x0, indices, isdict=True)

In [157]:
xfp = Struct.config(parallel=[Struct1, Struct2]).build(*set_info)(x)

In [158]:
print_formatted_table([xfp], indices, idxrev)

C_f    C_D    C_L    A     Re     W        S     V     Ww       D     
0.00e0 0.00e0 0.00e0 18.16 0.00e0 7083.108 5.256 49.18 2143.108 0.00e0


In [152]:
xfp = Aero.build(*set_info)(x)

In [153]:
print_formatted_table([xfp], indices, idxrev)

C_f      C_D  C_L   A  Re     W    S V  Ww     D      
4.19e-03 0.03 0.911 20 1.73e6 7000 5 50 0.00e0 231.926


In [159]:
# Config 1
C1 = AeroStruct.config(elim=[Struct.config(parallel=[Struct1, Struct2]), Aero]).build(*set_info)

In [160]:
xfp = C1(x)

In [161]:
idxrev = {var.item():key for key,var in indices.items()}
print_formatted_table([xfp], indices, idxrev)

C_f      C_D   C_L   A     Re     W        S     V     Ww       D      
4.14e-03 0.031 0.906 18.16 1.83e6 7083.108 5.256 49.18 2143.108 242.334


In [ ]:
# Config 2
C2 = AeroStruct.config(elim=[Aero, Struct1], residual=[Struct2]).build()